In [1]:
import pandas as pd 
import numpy as np 
import os 
import re 
import gc 
import os 
import cv2
import tqdm
import matplotlib.pyplot as plt
DATA_PATH = 'data/'

In [2]:
FILES = os.listdir(DATA_PATH)
FILES = [os.path.join(DATA_PATH,file) for file in FILES]
print(FILES)

['data/train_image_data_3.parquet', 'data/test_image_data_0.parquet', 'data/class_map.csv', 'data/test_image_data_2.parquet', 'data/bengaliai-cv19.zip', 'data/sample_submission.csv', 'data/train_image_data_0.parquet', 'data/train.csv', 'data/train_image_data_1.parquet', 'data/train_image_data_2.parquet', 'data/test.csv', 'data/test_image_data_3.parquet', 'data/test_image_data_1.parquet']


In [3]:
# Data load 
labels = ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']
train_data = pd.read_csv('data/train.csv')

In [4]:
# Train data desc
train_data.shape
train_data.columns

Index(['image_id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic',
       'grapheme'],
      dtype='object')

In [5]:
# Grapheme root unique 
root_uq = len(np.unique(train_data.grapheme_root))
#Vowel diactric 
vow_uq = len(np.unique(train_data.vowel_diacritic))
#Consonant diactritic 
con_uq = len(np.unique(train_data.consonant_diacritic))

In [6]:
print("Unique ROOT UQ {}".format(root_uq))
print("Unique VOW  UQ {}".format(vow_uq))
print("Unique CON  UQ {}".format(con_uq))

Unique ROOT UQ 168
Unique VOW  UQ 11
Unique CON  UQ 7


In [7]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    for i in range(df.shape[0]):
        image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

In [8]:
#train_image_0 = pd.read_parquet('data/train_image_data_0.parquet')

In [9]:
# Install additional packages
#!pip install pyarrow
#!pip install fastparquet

In [ ]:
b_train_data = pd.merge(pd.read_parquet('data/train_image_data_0.parquet'), 
                       train_data, on = 'image_id').drop(['image_id'], axis=1)

In [ ]:
train_image = b_train_data.drop(labels, axis = 1)

In [ ]:
HEIGHT, WIDTH =  137, 236 
images = train_data.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH)

In [ ]:
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=64, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    return cv2.resize(img,(size,size))